In [1]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from datetime import datetime
import pyspark.sql.functions as func
#导入需要的库
def toDate(inputStr):   #处理时间的函数
    newStr = ""
    if len(inputStr) == 8:
        s1 = inputStr[0:4]
        s2 = inputStr[5:6]
        s3 = inputStr[7]
        newStr = s1+"-"+"0"+s2+"-"+"0"+s3
    else:
        s1 = inputStr[0:4]
        s2 = inputStr[5:6]
        s3 = inputStr[7:]
        newStr = s1+"-"+"0"+s2+"-"+s3
    date = datetime.strptime(newStr, "%Y-%m-%d")
    return date
 

    #主程序
spark = SparkSession.builder.config(conf = SparkConf()).getOrCreate() #sparkSession参数设置
 
fields = [StructField("date", DateType(),False),StructField("county", StringType(),False),StructField("state", StringType(),False),
                    StructField("cases", IntegerType(),False),StructField("deaths", IntegerType(),False),]
schema = StructType(fields) #构建schema
 
rdd0 = spark.sparkContext.textFile("/user/hadoop/us.txt") #读取hdfs上转换为txt文件格式的数据集
rdd1 = rdd0.map(lambda x:x.split("\t")).map(lambda p: Row(toDate(p[0]),p[1],p[2],int(p[3]),int(p[4])))
 
shemaUsInfo = spark.createDataFrame(rdd1,schema) #读取文件生成DataFrame
 
shemaUsInfo.createOrReplaceTempView("usInfo") #将一个DataFrame表注册为一个名为usInfo的临时表，用于下面数据分析
 

In [11]:
#1.计算每日的累计确诊病例数和死亡数
df = shemaUsInfo.groupBy("date").agg(func.sum("cases"),func.sum("deaths")).sort(shemaUsInfo["date"].asc())
 
#列重命名
df1 = df.withColumnRenamed("sum(cases)","cases").withColumnRenamed("sum(deaths)","deaths")
df1.show()
df1.repartition(1).write.json("r1.json")                               #写入hdfs
 
#注册为临时表
df1.createOrReplaceTempView("ustotal")

+----------+-----+------+
|      date|cases|deaths|
+----------+-----+------+
|2020-01-21|    1|     0|
|2020-01-22|    1|     0|
|2020-01-23|    1|     0|
|2020-01-24|    2|     0|
|2020-01-25|    3|     0|
|2020-01-26|    5|     0|
|2020-01-27|    5|     0|
|2020-01-28|    5|     0|
|2020-01-29|    5|     0|
|2020-01-30|    6|     0|
|2020-01-31|    7|     0|
|2020-02-01|    8|     0|
|2020-02-02|   11|     0|
|2020-02-03|   11|     0|
|2020-02-04|   11|     0|
|2020-02-05|   12|     0|
|2020-02-06|   12|     0|
|2020-02-07|   12|     0|
|2020-02-08|   12|     0|
|2020-02-09|   12|     0|
+----------+-----+------+
only showing top 20 rows



In [3]:
#2.计算每日较昨日的新增确诊病例数和死亡病例数
df2 = spark.sql("select t1.date,t1.cases-t2.cases as caseIncrease,t1.deaths-t2.deaths as deathIncrease from ustotal t1,ustotal t2 where t1.date = date_add(t2.date,1)")
df2.show()
df2.sort(df2["date"].asc()).repartition(1).write.json("r2.json")           #写入hdfs

+----------+------------+-------------+
|      date|caseIncrease|deathIncrease|
+----------+------------+-------------+
|2020-04-30|       30425|         2211|
|2020-03-07|         117|            4|
|2020-03-13|         556|            7|
|2020-02-04|           0|            0|
|2020-02-15|           0|            0|
|2020-02-12|           1|            0|
|2020-05-08|       27654|         1573|
|2020-04-29|       26510|         2514|
|2020-05-10|       20159|          931|
|2020-01-25|           1|            0|
|2020-04-26|       26598|         1246|
|2020-04-21|       25488|         2668|
|2020-03-09|         201|            4|
|2020-03-10|         270|            5|
|2020-03-11|         245|            6|
|2020-04-19|       25354|         1518|
|2020-02-11|           0|            0|
|2020-04-13|       25787|         1762|
|2020-05-17|       18985|          844|
|2020-02-25|           2|            0|
+----------+------------+-------------+
only showing top 20 rows



In [4]:
#3.统计截止5.19日 美国各州的累计确诊人数和死亡人数
df3 = spark.sql("select date,state,sum(cases) as totalCases,sum(deaths) as totalDeaths,round(sum(deaths)/sum(cases),4) as deathRate from usInfo  where date = date('2020-05-19') group by date,state")
df3.show()
df3.sort(df3["totalCases"].desc()).repartition(1).write.json("r3.json") #写入hdfs
 
df3.createOrReplaceTempView("eachStateInfo")

+----------+--------------------+----------+-----------+---------+
|      date|               state|totalCases|totalDeaths|deathRate|
+----------+--------------------+----------+-----------+---------+
|2020-05-19|Northern Mariana ...|        21|          2|   0.0952|
|2020-05-19|            Oklahoma|      5489|        294|   0.0536|
|2020-05-19|         Connecticut|     38430|       3472|   0.0903|
|2020-05-19|             Florida|     46936|       2051|   0.0437|
|2020-05-19|District of Columbia|      7434|        400|   0.0538|
|2020-05-19|               Maine|      1741|         73|   0.0419|
|2020-05-19|           Wisconsin|     13001|        467|   0.0359|
|2020-05-19|              Hawaii|       631|         17|   0.0269|
|2020-05-19|          California|     83981|       3422|   0.0407|
|2020-05-19|      Virgin Islands|        69|          6|    0.087|
|2020-05-19|            Colorado|     22454|       1257|    0.056|
|2020-05-19|              Nevada|      7170|        373|    0.

In [5]:
#4.找出美国确诊最多的10个州
df4 = spark.sql("select date,state,totalCases from eachStateInfo  order by totalCases desc limit 10")
df4.show()
df4.repartition(1).write.json("r4.json")
df4.createOrReplaceTempView("MostStateInfo")

+----------+-------------+----------+
|      date|        state|totalCases|
+----------+-------------+----------+
|2020-05-19|     New York|    357757|
|2020-05-19|   New Jersey|    149037|
|2020-05-19|     Illinois|     98298|
|2020-05-19|Massachusetts|     87925|
|2020-05-19|   California|     83981|
|2020-05-19| Pennsylvania|     67404|
|2020-05-19|     Michigan|     52337|
|2020-05-19|        Texas|     51080|
|2020-05-19|      Florida|     46936|
|2020-05-19|     Maryland|     41664|
+----------+-------------+----------+



In [6]:
#5.找出美国死亡最多的10个州
df5 = spark.sql("select date,state,totalDeaths from eachStateInfo  order by totalDeaths desc limit 10")
df5.show()
df5.repartition(1).write.json("r5.json")
df5.createOrReplaceTempView("MostStateDeathsInfo")

+----------+-------------+-----------+
|      date|        state|totalDeaths|
+----------+-------------+-----------+
|2020-05-19|     New York|      28437|
|2020-05-19|   New Jersey|      10586|
|2020-05-19|Massachusetts|       5938|
|2020-05-19|     Michigan|       5017|
|2020-05-19| Pennsylvania|       4675|
|2020-05-19|     Illinois|       4399|
|2020-05-19|  Connecticut|       3472|
|2020-05-19|   California|       3422|
|2020-05-19|    Louisiana|       2581|
|2020-05-19|     Maryland|       2081|
+----------+-------------+-----------+



In [7]:
#6.找出美国确诊最少的10个州
df6 = spark.sql("select date,state,totalCases from eachStateInfo  order by totalCases asc limit 10")
df6.show()
df6.repartition(1).write.json("r6.json")
df6.createOrReplaceTempView("leastStateInfo")

+----------+--------------------+----------+
|      date|               state|totalCases|
+----------+--------------------+----------+
|2020-05-19|Northern Mariana ...|        21|
|2020-05-19|      Virgin Islands|        69|
|2020-05-19|              Alaska|       399|
|2020-05-19|             Montana|       471|
|2020-05-19|              Hawaii|       631|
|2020-05-19|             Wyoming|       776|
|2020-05-19|             Vermont|       944|
|2020-05-19|                Guam|      1123|
|2020-05-19|       West Virginia|      1514|
|2020-05-19|               Maine|      1741|
+----------+--------------------+----------+



In [8]:
#7.找出美国死亡最少的10个州
df7 = spark.sql("select date,state,totalDeaths from eachStateInfo  order by totalDeaths asc limit 10")
df7.show()
df7.repartition(1).write.json("r7.json")
df7.createOrReplaceTempView("leastStateDeathsInfo")

+----------+--------------------+-----------+
|      date|               state|totalDeaths|
+----------+--------------------+-----------+
|2020-05-19|Northern Mariana ...|          2|
|2020-05-19|      Virgin Islands|          6|
|2020-05-19|                Guam|          6|
|2020-05-19|              Alaska|          8|
|2020-05-19|             Wyoming|         10|
|2020-05-19|             Montana|         16|
|2020-05-19|              Hawaii|         17|
|2020-05-19|        North Dakota|         45|
|2020-05-19|        South Dakota|         46|
|2020-05-19|             Vermont|         54|
+----------+--------------------+-----------+



In [9]:
#8.统计截止5.19全美和各州的病死率
df8 = spark.sql("select 1 as sign,date,'USA' as state,round(sum(totalDeaths)/sum(totalCases),4) as deathRate from eachStateInfo group by date union select 2 as sign,date,state,deathRate from eachStateInfo").cache()
df8.show()
df8.sort(df8["sign"].asc(),df8["deathRate"].desc()).repartition(1).write.json("r8.json")
df8.createOrReplaceTempView("USAStateDeathsInfo")

+----+----------+-------------+---------+
|sign|      date|        state|deathRate|
+----+----------+-------------+---------+
|   2|2020-05-19|     Virginia|   0.0324|
|   2|2020-05-19|      Arizona|   0.0483|
|   2|2020-05-19| North Dakota|   0.0226|
|   2|2020-05-19|Massachusetts|   0.0675|
|   2|2020-05-19|   New Mexico|   0.0446|
|   2|2020-05-19|        Idaho|   0.0311|
|   2|2020-05-19|     Oklahoma|   0.0536|
|   2|2020-05-19|     Kentucky|    0.045|
|   2|2020-05-19|   California|   0.0407|
|   2|2020-05-19|         Utah|   0.0117|
|   2|2020-05-19| South Dakota|   0.0113|
|   2|2020-05-19|West Virginia|   0.0449|
|   2|2020-05-19|     Illinois|   0.0448|
|   2|2020-05-19|        Texas|   0.0275|
|   2|2020-05-19|       Oregon|   0.0376|
|   2|2020-05-19|     Maryland|   0.0499|
|   2|2020-05-19| Rhode Island|   0.0411|
|   2|2020-05-19|       Nevada|    0.052|
|   2|2020-05-19|       Hawaii|   0.0269|
|   2|2020-05-19|     Arkansas|   0.0207|
+----+----------+-------------+---